In [20]:
#importing libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report


In [21]:
data=pd.read_csv('/content/mobile_price_classification.csv')

In [22]:
# Separate features and target variable
X = data.drop(columns=["price_range"])
y = data["price_range"]


In [23]:
y = pd.get_dummies(y).values

In [24]:
#Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
#Scaling features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#ANN

Install Required Libraries: Ensure you have TensorFlow and Keras installed. You can install them using pip:

In [26]:
!pip install tensorflow keras

In [27]:
def build_model(learning_rate=0.001, dropout_rate=0.2):
    model = Sequential([
        Dense(64, input_dim=X_train.shape[1], activation="relu"),
        Dropout(dropout_rate),
        Dense(32, activation="relu"),
        Dense(y.shape[1], activation="softmax")
    ])
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    return model


#Hyperparameter Tuning

Use a grid search or random search to optimize hyperparameters like:

* Number of neurons in hidden layers

* Learning rate
* Batch size


* Number of epochs

In [28]:
learning_rates = [0.001, 0.01]
dropout_rates = [0.2, 0.3]

best_model = None
best_accuracy = 0

for lr in learning_rates:
    for dr in dropout_rates:
        print(f"Training with learning rate={lr}, dropout_rate={dr}")
        model = build_model(learning_rate=lr, dropout_rate=dr)
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32, verbose=0)
        _, accuracy = model.evaluate(X_test, y_test, verbose=0)

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model

Training with learning rate=0.001, dropout_rate=0.2
Training with learning rate=0.001, dropout_rate=0.3
Training with learning rate=0.01, dropout_rate=0.2
Training with learning rate=0.01, dropout_rate=0.3


In [29]:
#Evaluate the best model
print(f"Best Accuracy: {best_accuracy}")
y_pred = np.argmax(best_model.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)

Best Accuracy: 0.9325000047683716
13/13 [==============================] - 0s 1ms/step


In [30]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95       105
           1       0.91      0.93      0.92        91
           2       0.93      0.88      0.91        92
           3       0.94      0.95      0.94       112

    accuracy                           0.93       400
   macro avg       0.93      0.93      0.93       400
weighted avg       0.93      0.93      0.93       400



In [31]:
best_model.save("mobile_price_classifier.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
